<a href="https://colab.research.google.com/github/YanSym/Amazonia_Azul_Reporter/blob/master/ML_Olympiad_Quality_Education.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Olympiad - Quality Education

<center><img src="https://guiadoestudante.abril.com.br/wp-content/uploads/sites/4/2021/11/Provas.jpg" alt="enem" width="400"></center>


## Import das bibliotecas utilizadas

In [ ]:
import pandas as pd
import numpy as np
from pylab import rcParams
from google.colab import drive
from functools import reduce
import warnings
from tqdm import tqdm
from math import sqrt
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import LabelEncoder
import gc

# visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.gofplots import qqplot

# coordenadas geográficas
!pip install geobr
import geopandas as gpd
import geobr

# drive do Google Colab
drive.mount('/content/gdrive')

# Parâmetros
plt.rcParams['figure.figsize'] = [8, 8]
pd.set_option("display.max_columns", 200)
pd.set_option('display.max_rows', 200)
plt.style.use('ggplot')

# Filtra warnings
warnings.filterwarnings("ignore")

# Plot de gráficos
%matplotlib inline

geobr.read_state(year = 2019)

     |████████████████████████████████| 928 kB 4.2 MB/s 
     |████████████████████████████████| 16.7 MB 300 kB/s 
     |████████████████████████████████| 6.3 MB 46.4 MB/s 
Mounted at /content/gdrive


,code_state,abbrev_state,name_state,code_region,name_region,geometry
0,11.0,RO,Rondônia,1.0,Norte,"MULTIPOLYGON (((-65.38150 -10.42907, -65.37916..."
1,12.0,AC,Acre,1.0,Norte,"MULTIPOLYGON (((-71.07772 -9.82774, -71.08554 ..."
2,13.0,AM,Amazônas,1.0,Norte,"MULTIPOLYGON (((-69.83766 -3.68659, -69.71230 ..."
3,14.0,RR,Roraima,1.0,Norte,"MULTIPOLYGON (((-63.96008 2.47313, -63.96041 2..."
4,15.0,PA,Pará,1.0,Norte,"MULTIPOLYGON (((-51.43248 -0.47334, -51.42843 ..."
5,16.0,AP,Amapá,1.0,Norte,"MULTIPOLYGON (((-50.45011 2.10924, -50.44252 2..."
6,17.0,TO,Tocantins,1.0,Norte,"MULTIPOLYGON (((-48.23163 -5.94614, -48.22940 ..."
7,21.0,MA,Maranhão,2.0,Nordeste,"MULTIPOLYGON (((-44.53830 -2.02124, -44.53542 ..."
8,22.0,PI,Piauí,2.0,Nordeste,"MULTIPOLYGON (((-42.91539 -6.64607, -42.91722 ..."
9,23.0,CE,Ceará,2.0,Nordeste,"MULTIPOLYGON (((-41.18292 -4.01797, -41.18862 ..."


### Variáveis Globais

In [ ]:
!pip install geobr

In [ ]:
# Divisão das Variaveis
lista_targets = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']
Variaveis_Socio_Economico = ['Q001','Q002',	'Q003',	'Q004',	'Q005',	'Q006',	'Q007',	'Q008',	'Q009',	'Q010',	'Q011',	'Q012',	'Q013',	'Q014',	'Q015',	'Q016',	'Q017',	'Q018',	'Q019',	'Q020',	'Q021',	'Q022',	'Q023',	'Q024',	'Q025']
Variaveis_Atendimento_Especializado = ['IN_BAIXA_VISAO',	'IN_CEGUEIRA',	'IN_SURDEZ',	'IN_DEFICIENCIA_AUDITIVA',	'IN_SURDO_CEGUEIRA',	'IN_DEFICIENCIA_FISICA',	'IN_DEFICIENCIA_MENTAL',	'IN_DEFICIT_ATENCAO',	'IN_DISLEXIA',	'IN_DISCALCULIA',	'IN_AUTISMO',	'IN_VISAO_MONOCULAR',	'IN_OUTRA_DEF']
Variaveis_Cadastrais = ['NU_INSCRICAO',	'CO_MUNICIPIO_RESIDENCIA',	'NO_MUNICIPIO_RESIDENCIA',	'CO_UF_RESIDENCIA',	'SG_UF_RESIDENCIA',	'NU_IDADE',	'TP_SEXO',	'TP_ESTADO_CIVIL',	'TP_COR_RACA',	'TP_NACIONALIDADE',	'CO_MUNICIPIO_NASCIMENTO',	'NO_MUNICIPIO_NASCIMENTO',	'CO_UF_NASCIMENTO',	'SG_UF_NASCIMENTO',	'TP_ST_CONCLUSAO',	'TP_ANO_CONCLUIU',	'TP_ESCOLA',	'TP_ENSINO',	'IN_TREINEIRO']

## Métodos Auxiliares

In [ ]:
def reduce_mem_usage(df):
    """
    iterate through all the columns of a dataframe and modify the data type to reduce memory usage.
    fonte: https://www.kaggle.com/valleyzw/ubiquant-lgbm-baseline  
    """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in tqdm([x for x in df.columns if 'NU_NOTA_' not in x]):
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
 

def show_missings(df):
    '''
    mostra porcentagem de missings no dataframe
    '''
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                     'percent_missing': percent_missing})

    missing_value_df = missing_value_df.sort_values('percent_missing', ascending=False).round(2)
    return (missing_value_df)


def leitura_dados(path_train, path_test, sample_frac=-1):
    '''
    método para leitura dos datasets
    '''
    if sample_frac == -1:
      df_desenv = reduce_mem_usage(pd.read_csv(path_train))
    else:
      df_desenv = reduce_mem_usage(pd.read_csv(path_train).sample(frac=sample_frac))
    df_submit = reduce_mem_usage(pd.read_csv(path_test))
    return (df_desenv, df_submit)
        

def rmse_score(true, pred):
    '''
    rmse score
    '''
    return (sqrt(mean_squared_error(true, pred)))


def map_cor_raca(cor_raca):
    '''
    mapeia cor/raca de acordo com o metadados fornecido
    '''
    if cor_raca == 0:
        return 'Não informado'
    elif cor_raca == 1:
        return 'Branca'
    elif cor_raca == 2:
        return 'Preta'
    elif cor_raca == 3:
        return 'Parda'
    elif cor_raca == 4:
        return 'Amarela'
    elif cor_raca == 5:
        return 'Indígena'
    else:
        return ''


def map_estado_civil(estado_civil):
    '''
    mapeia estado civil de acordo com o metadados fornecido
    '''
    if estado_civil == 0:
        return 'Não informado'
    elif estado_civil == 1:
        return 'Solteiro(a)'
    elif estado_civil == 2:
        return 'Casado(a)/Mora com companheiro(a)'
    elif estado_civil == 3:
        return 'Divorciado(a)/Desquitado(a)/Separado(a)'
    elif estado_civil == 4:
        return 'Viúvo(a)'
    else:
        return ''


def gerar_painel_barra(data_frame,
                      var,
                      hue,
                      title = '',
                      title_subplot_1 = '',
                      title_subplot_2 = '',
                      legend_subplot_2 = '',
                      xlabel = 'Quantidade',
                      ylabel = '',
                      figsize = (12, 6)
                      ):
  '''
  gera gráfico de barras
  '''
  fig, ax = plt.subplots(1, 2, figsize = figsize)
  sns.countplot(data = data_frame,
                y = var,
                ax = ax[0])
  sns.countplot(data = data_frame,
                y = var,
                hue = hue,
                ax = ax[1])
  ax[0].set(ylabel = ylabel, xlabel = xlabel, title = title_subplot_1)
  ax[1].set(ylabel = ylabel, xlabel = xlabel, title = title_subplot_2)
  ax[1].legend(title = legend_subplot_2)
  fig.suptitle(title)
  fig.tight_layout(pad = 4)


def print_importancias_lasso(df, coef):
  '''
  importância das variáveis explicativas do modelo lasso
  '''
  for e in sorted (list(zip(list(df), coef)), key = lambda e: -abs(e[1])):
    if e[1] != 0:
      print('\t{}, {:.3f}'.format(e[0], e[1]))


def percentile(n):
    '''
    retorna percentil
    '''
    def percentile_(x):
        return x.quantile(n)
    percentile_.__name__ = 'percentile_{:2.0f}'.format(n*100)
    return percentile_


def estatistica_descritiva_por_estado(df, metrica, localizacao_estado = geobr.read_state(year = 2019)):
  "Calcula alguma estatística descritiva para as notas do Enem por estado."
  # provas do dataset de base
  provas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']
  # obtém os resultados por estado conforme medida estatística inserida
  df = df.groupby(by = 'SG_UF_RESIDENCIA', as_index = False)[provas].agg(metrica)
  # geolocalização
  df = gpd.GeoDataFrame(pd.merge(
    df,
    localizacao_estado,
    left_on = 'SG_UF_RESIDENCIA',
    right_on = 'abbrev_state',
    how = 'inner'))
  
  return df

def plot_mapa_estado(df, estatistica_descritiva = np.mean, title = '', cmap = 'BuPu'):
  '''
  gera mapa heatmap para o Brasil populado com a estatística descritiva de interesse
  '''
  # cria o DataFrame conforme estatística descritiva definida
  df = estatistica_descritiva_por_estado(df=df, metrica = estatistica_descritiva)
  # labels para o pllot
  labels_provas = ['Ciências da Natureza', 'Ciências Humanas', 'Linguagens', 'Matemática', 'Redação']
  # colunas referentes a prova
  provas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']
  # cria a figura
  fig, ax = plt.subplots(1, 5, figsize = (20, 20))
  # itera na lista de provas e cria o mapa
  for index, prova in enumerate(provas):
    df.plot(
        column = prova,
        cmap = cmap,
        edgecolor = 'lightgray',
        lw = 0.3,
        ax = ax[index],
        legend=True,
        legend_kwds = {'shrink': 0.08}
        )
    # remove marcações dos eixos
    ax[index].axis('off')
    # labels
    ax[index].set_title(labels_provas[index], fontsize  = 10)

  fig.suptitle(title, y = 0.6 , weight = 'bold')
  fig.tight_layout(pad = 2);


def Gerar_Grafico_Economico(Coluna):
  '''
  Função para gerar gráfico socio economico
  '''

  # Posicao
  Posicao_Index = 1

  # Ordenando os dados
  Filtro = Analise_Econominca.sort_values(by=str(Coluna))

  # Tamanho da Imagem
  fig, ax = plt.subplots(figsize=(18, 15))

  # Cor de fundo
  Cor_Fundo = "#F5F4EF"
  ax.set_facecolor(Cor_Fundo)
  fig.set_facecolor(Cor_Fundo)

  # Paleta de Cores
  Paleta_Cores = sns.color_palette('flare', 7)

  # Estilo do gráfico
  plt.style.use('seaborn-darkgrid')

  # Incluindo o Titulo na Figura
  plt.suptitle(f'Dados do Questionário Socieconômico | {Coluna}', 
             fontsize=22, color='#404040', fontfamily='KyivType Sans', fontweight=600 )

  # Loop plotar os gráficos
  for Grafico in Analise_Econominca[lista_targets]:

    # Retiando qlq valor zerado
    Filtro = Filtro.loc[ Filtro[Grafico] > 0 ]

    # Posição do Plot
    plt.subplot( 5, 1, Posicao_Index )

    # Plot
    sns.boxplot( data=Filtro, x=str(Coluna), y=Grafico, showmeans = True, linewidth=1, width=0.4, color=Paleta_Cores[Posicao_Index] )
    sns.stripplot( data=Filtro, x=str(Coluna), y=Grafico, size=0.3, color='0.1', linewidth=0 )

    # Titulo
    plt.title(f'Análise {Grafico}', loc='left', fontsize=14, fontweight=200)

    # Labels
    plt.ylabel('Nota de 0 - 1000')
    plt.xlabel('Resposta dos participantes')

    # Ajustando distancias dos gráficos no relatorio
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.94, wspace=0.2, hspace=0.25);

    # Troca index do Gráfico
    Posicao_Index += 1

URLError: ignored

## Leitura de Dados

##### Dataset de desenvolvimento do modelo

In [ ]:
path_input_train='/content/gdrive/MyDrive/ML_Olympiad_Kaggle/train.csv'
path_input_test='/content/gdrive/MyDrive/ML_Olympiad_Kaggle/test.csv'
path_output_submission='/content/gdrive/MyDrive/ML_Olympiad_Kaggle/df_submit_baseline.csv'

df_desenv, df_submit = leitura_dados(path_input_train, path_input_test, sample_frac=0.1)
print (df_desenv.shape, df_submit.shape)

In [ ]:
df_desenv.head()

In [ ]:
df_submit.head()

## Análise Exploratória de Dados

In [ ]:
# descritivo das variáveis do dataset
df_desenv.describe()

In [ ]:
df_desenv.info()

In [ ]:
# tipo das variáveis do dataset
df_desenv.dtypes

In [ ]:
# lista de variáveis do dataset
df_desenv.columns.tolist()

In [ ]:
# colunas do target
df_desenv[lista_targets].head()

In [ ]:
# porcentagem de dados faltantes (missings) no dataset
show_missings(df_desenv)

In [ ]:
# quantidade de valores únicos em cada coluna
df_desenv.nunique()

In [ ]:
# frequência dos inscritos no Enem por UF
# Através das análises, é possível observar que a maioria dos inscritos é de São Paulo (16%), seguido de Minas Gerais (10,5%) e Bahia (7,7%)
df_desenv['SG_UF_RESIDENCIA'].value_counts(dropna=False, ascending=False, normalize=True)

In [ ]:
# Correlação entre as notas das provas do Enem
fig, ax = plt.subplots()
corr_provas = df_desenv[lista_targets].corr()
sns.heatmap(corr_provas, annot=True, cmap = 'Blues', ax = ax);

### Análises Visuais

In [ ]:
# Boxplot das notas do Enem
fig, ax = plt.subplots(figsize = (10, 10))
sns.boxplot(data = df_desenv[lista_targets], ax = ax);

In [ ]:
# Analisando Residencia x Notas
Analise_Target = df_desenv[['SG_UF_RESIDENCIA', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']]

# Criando o relátorio
fig, axs = plt.subplots(3, 2, figsize=(15, 10))

# Plotando as notas em histogramas
sns.histplot(data=Analise_Target, x='NU_NOTA_CN', color='skyblue', bins=100, ax=axs[0, 0])
sns.histplot(data=Analise_Target, x='NU_NOTA_CH', color='olive', bins=100, ax=axs[0, 1])
sns.histplot(data=Analise_Target, x='NU_NOTA_LC', color='gold', bins=100, ax=axs[1, 0])
sns.histplot(data=Analise_Target, x='NU_NOTA_MT', color='teal', bins=100, ax=axs[1, 1])
sns.histplot(data=Analise_Target, x='NU_NOTA_REDACAO', color='olive', bins=50, ax=axs[2, 0])
axs[2, 1].set_axis_off()

# Incluindo o Titulo na Figura
plt.suptitle('Distribuição das notas para cada prova', fontsize=22, color='#404040', fontweight=600);

In [ ]:
print ('Idade média:', round(df_desenv['NU_IDADE'].mean(), 2))
print ('Idade mediana:', round(df_desenv['NU_IDADE'].median(), 2))
print ('Idade mais frequente (moda):', round(df_desenv['NU_IDADE'].mode()[0], 2))
plt.hist(df_desenv['NU_IDADE'], bins=50);
plt.title('Perfil etário dos inscritos no Enem');

In [ ]:
sns.countplot(x='TP_SEXO', data=df_desenv)
plt.title('Distribuição de Gênero dos inscritos');

In [ ]:
df_desenv['MAP_TP_COR_RACA'] = df_desenv['TP_COR_RACA'].apply(map_cor_raca)
gerar_painel_barra(df_desenv, 'MAP_TP_COR_RACA', 'TP_SEXO', 
                   title = 'Perfil de cor e raça dos inscritos',
                   title_subplot_1 = 'Cor/raça',
                   title_subplot_2 = 'Cor/raça por gênero',
                   legend_subplot_2 = 'Gênero',
                   ylabel = 'Cor/raça')

In [ ]:
df_desenv['MAP_TP_ESTADO_CIVIL'] = df_desenv['TP_ESTADO_CIVIL'].apply(map_estado_civil)
        
fig, ax = plt.subplots(1,2, figsize = (12, 6))
sns.countplot(data = df_desenv,
             y = 'MAP_TP_ESTADO_CIVIL', ax = ax[0])
sns.countplot(data = df_desenv,
             y = 'MAP_TP_ESTADO_CIVIL', hue = 'TP_SEXO', ax = ax[1])
ax[0].set(ylabel = 'Estado Civil', xlabel = 'Quantidade',title = 'Estado civil')
ax[1].set(ylabel = 'Estado Civil', xlabel = 'Quantidade',title = 'Estado civil por gênero')
ax[1].legend(title = 'Gênero')
fig.suptitle('Estado civil dos inscritos')
fig.tight_layout(pad = 4);

In [ ]:
# Calcula a quantidade de inscritos em cada estado (amostra)
# Através dessa análise, podemos concluir que a maria dos inscritos é de São Paulo
df_inscritos_por_estado = df_desenv.groupby(by = 'SG_UF_RESIDENCIA')[['SG_UF_RESIDENCIA']].count()\
.rename(columns = {'SG_UF_RESIDENCIA': 'quantidade_inscritos'})\
.reset_index()\
.sort_values(by = 'quantidade_inscritos', ascending = False)

# inscritos por estado com geolocalização
df_inscritos_por_estado_spatial_data = gpd.GeoDataFrame(pd.merge(
    df_inscritos_por_estado,
    geobr.read_state(year = 2019),
    left_on = 'SG_UF_RESIDENCIA',
    right_on = 'abbrev_state',
    how = 'inner'
))

fig, ax = plt.subplots(figsize = (10, 8))
df_inscritos_por_estado_spatial_data.plot(
    column = 'quantidade_inscritos',
    cmap = 'Blues',
    legend = True,
    legend_kwds= {
        "label": "Quantidade de inscritos",
        "orientation": "horizontal",
        "shrink": 0.4,
    },
    edgecolor = 'lightgray',
    lw = 0.3,
    ax = ax
)

ax.set_title('Quantidade de inscritos no Enem por estado')
ax.axis('off');

In [ ]:
# Calcula a nota média de cada prova por estado
# Através dessa análise, podemos concluir as regiões Sul e Sudeste possuem as maiores notas médias, e a região Norte as menores
plot_mapa_estado(df_desenv, np.mean, 'Média das notas por estado', 'hot_r')

In [ ]:
# Calcula a nota percentil 99 de cada prova por estado
# Através dessa análise, podemos concluir a região Sudeste possuem as maiores notas do percentil 99, e a região Norte as menores
plot_mapa_estado(df_desenv, percentile(0.99), 'Percentil 99 das notas por estado', 'hot_r')

In [ ]:
print('Até que série seu pai, ou o homem responsável por você, estudou? \n')
Analise_Econominca = df_desenv[Variaveis_Socio_Economico + lista_targets] 
Gerar_Grafico_Economico('Q001')

In [ ]:
print('Até que série sua mãe, ou a mulher responsável por você, estudou? \n')
Gerar_Grafico_Economico('Q002')

In [ ]:
print('A partir da apresentação de algumas ocupações divididas em grupos ordenados, indique o grupo que contempla a ocupação mais próxima da ocupação do seu pai ou do homem responsável por você. (Se ele não estiver trabalhando, escolha uma ocupação pensando no último trabalho dele). \n')
Gerar_Grafico_Economico('Q003')

In [ ]:
print('A partir da apresentação de algumas ocupações divididas em grupos ordenados, \n indique o grupo que contempla a ocupação mais próxima da ocupação da sua mãe ou da mulher responsável por você. (Se ela não estiver trabalhando, \n escolha uma ocupação pensando no último trabalho dela). \n')
Gerar_Grafico_Economico('Q004')

In [ ]:
print('Incluindo você, quantas pessoas moram atualmente em sua residência? \n')
Gerar_Grafico_Economico('Q005')

In [ ]:
print('Qual é a renda mensal de sua família? (Some a sua renda com a dos seus familiares.) \n')
Gerar_Grafico_Economico('Q006')

In [ ]:
# Analisando Residência x Notas na prova do Enem
Analise_UF = df_desenv[['SG_UF_RESIDENCIA', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']]

# Tamanho da Imagem
fig, ax = plt.subplots(figsize=(20, 10))

# Cor de fundo
Cor_Fundo = "#F5F4EF"
ax.set_facecolor(Cor_Fundo)
fig.set_facecolor(Cor_Fundo)

# Estilo do gráfico
plt.style.use('seaborn-darkgrid')

# Posição do Plot
plt.subplot(2, 1, 1)

# Plot
plt.plot(Analise_UF.groupby( by='SG_UF_RESIDENCIA').median(), linewidth=4, alpha=0.7)

# Titulo
plt.title('Mediana das provas por Estado', loc='left', fontsize=14, fontweight=0)

# Labels
plt.xlabel('Estados', fontsize=14 )
plt.ylabel('Nota de 0 - 1000', fontsize=14)

# Legenda
plt.legend( ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO'], 
           ncol=5, fontsize=12, loc='upper center', bbox_to_anchor=(0.5, -0.1), shadow=True )

# Posição do Plot
plt.subplot( 2, 1, 2 )

# Plot
plt.plot( Analise_UF.groupby( by='SG_UF_RESIDENCIA').mean(), linewidth=4, alpha=0.7 )

# Titulo
plt.title('Média das provas por Estado', loc='left', fontsize=14, fontweight=0)

# Labels
plt.xlabel('Estados', fontsize=14)
plt.ylabel('Nota de 0 - 1000', fontsize=14)

# Legenda
plt.legend( ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO'], 
           ncol=5, fontsize=12, loc='upper center', bbox_to_anchor=(0.5, -0.1), shadow=True )

# Ajustando distancias dos gráficos no relatorio
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.91, wspace=0.2, hspace=0.4)

# Incluindo o Titulo na Figura
plt.suptitle('Média/Media por Estado do participante', 
             fontsize=22, color='#404040', fontfamily='KyivType Sans', fontweight=600 );

In [ ]:
# Tamanho da Imagem
fig, ax = plt.subplots(figsize=(20, 25))

# Cor de fundo
Cor_Fundo = "#F5F4EF"
ax.set_facecolor(Cor_Fundo)
fig.set_facecolor(Cor_Fundo)

# Estilo do gráfico
plt.style.use('seaborn-darkgrid')

# Lista de Paletas de Cores
Paleta_Cores = sns.color_palette('flare', 30)

# Dimensão do relatório
Quantidade_Linhas = round (Analise_UF['SG_UF_RESIDENCIA'].nunique() / 4 ) + 1
Colunas = 4

# Loop para criar o gráfico
for Posicao, Estado in enumerate( sorted( list( Analise_UF['SG_UF_RESIDENCIA'].unique() ) ) + list(['Geral']) ):
    
    # Pisção do GRáfico
    plt.subplot( Quantidade_Linhas, Colunas, Posicao + 1 )
    
    # Condição
    if Estado != 'Geral':
        # Filtro do Estado
        Nota_Matematica = Analise_UF.loc[ 
            ( Analise_UF['SG_UF_RESIDENCIA'] == Estado ) & ( Analise_UF['NU_NOTA_MT'] > 0 ) ]['NU_NOTA_MT']
        
        # Titulo do gráfico
        plt.title(f'Nota de Matemática do Estado: {Estado}', loc='left', fontsize=14, fontweight=100 )
        
    else:
        # Filtro para todos Estados
        Nota_Matematica = Analise_UF.loc[ ( Analise_UF['NU_NOTA_MT'] > 0 ) ]['NU_NOTA_MT']
        
        # Titulo do gráfico
        plt.title(f'Nota de Matemática nível Brasil', loc='left', fontsize=14, fontweight=100 )
    
    # Plot do Boxplot
    sns.boxplot( Nota_Matematica, 
                saturation = 0.55, 
                color = Paleta_Cores[Posicao],
                showmeans = True,
                linewidth=1,
                width=0.5,
                meanprops = {'markerfacecolor':'green', 'markeredgecolor':'white', 'markersize':'8'} )
    
    # Anotação em cada gráficos
    plt.annotate(
        f'Média:{ round(Nota_Matematica.mean()) }',
        xy = (1.0, -0.18),
        xycoords='axes fraction',
        ha = 'right', 
        va = 'center', 
        color='green', 
        fontsize=12,
        bbox = dict( facecolor="#fff", edgecolor="#009000", boxstyle="round", pad=0.25 ),
        fontweight=500 )
    
    # Labels
    plt.xlabel('Nota de 0 - 1000')
    plt.ylabel('Distribuição das Notas')
    
    # Ajuste da escala
    plt.xlim(0, 1000)

# Ajustando espaços entre os gráficos
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.94, wspace=0.2, hspace=0.45)

# Incluindo o Titulo na Figura
plt.suptitle('Análise nota de Matemática por Estado - Exame Nacional do Ensino Médio (Enem)', 
             fontsize=22, 
             color='#404040', 
             fontfamily='KyivType Sans', 
             fontweight=600 );

### Testes de Hipótese

In [ ]:
# Análise da cor de pele: brancos e pretos
# Através dessa análise é possível observar como são diferentes as notas para pessoas de cor branca e preta
# Pessoas de cor branca possuem notas maiores do que pessoas pretas, tanto na média quanto no percentil 99
brancos = df_desenv.query("TP_COR_RACA == 1 and NU_NOTA_MT == NU_NOTA_MT")['NU_NOTA_MT']
pretos = df_desenv.query("TP_COR_RACA == 2 and NU_NOTA_MT == NU_NOTA_MT")['NU_NOTA_MT']

print (f'Nota média em matemática de inscritos brancos: {round(brancos.mean(), 2)}')
print (f'Nota média em matemática de inscritos pretos: {round(pretos.mean(), 2)}')
print (f'Percentil 99 da nota em matemática de inscritos brancos: {round(np.quantile(brancos, 0.99), 2)}')
print (f'Percentil 99 da nota em matemática de inscritos pretos: {round(np.quantile(pretos, 0.99), 2)}')

plt.hist(brancos, bins=30, label='Branco', histtype='stepfilled', color='green')
plt.hist(pretos, bins=30, label='Preto', alpha=.7, histtype='stepfilled', color='red')
plt.xlabel('Nota em Matemática')
plt.ylabel('Quantidade')
plt.axvline(x=brancos.mean(), color='limegreen')
plt.axvline(x=pretos.mean(), color='darkred')
plt.title('Distribuição das notas de Matemática dos inscritos brancos e pretos')
plt.legend()
plt.show()

# Boxplot das notas de Matemática de brancos e pretos
dict_notas = {'brancos': brancos.values.tolist(),
              'pretos': pretos.values.tolist()}
fig, ax = plt.subplots(figsize = (10, 10))
ax.boxplot(dict_notas.values())
ax.set_xticklabels(dict_notas.keys())
plt.title('Boxplot das notas de Matemática dos inscritos brancos e pretos')
plt.show()

# Teste de hipótese
# Como o p-valor é de 0, devemos rejeitar a hipótese nula (de que não existe correlação entre a cor do inscrito e a sua nota de Matemática no Enem)
# Desse modo, é possível concluir que a cor está correlacionada com a nota de Matemática do Enem
stats.ks_2samp(brancos, pretos)

In [ ]:
# Análise da nota de matemática de inscritos treineiros e não treineiros
# Através dessa análise é possível observar como são diferentes as notas para treineiros e não treineiros
# Curiosamente, treineiros possuem uma nota média maior do que não treineiros,
# mas a nota percentil 99 dos não treineiros é muito superior à nota dos treineiros
treineiros = df_desenv.query("IN_TREINEIRO == 1 and NU_NOTA_MT == NU_NOTA_MT")['NU_NOTA_MT']
nao_treineiros = df_desenv.query("IN_TREINEIRO == 0 and NU_NOTA_MT == NU_NOTA_MT")['NU_NOTA_MT']

print (f'Nota média em matemática de inscritos treineiros: {round(treineiros.mean(), 2)}')
print (f'Nota média em matemática de inscritos não treineiros: {round(nao_treineiros.mean(), 2)}')
print (f'Percentil 99 da nota em matemática de inscritos treineiros: {round(np.quantile(treineiros, 0.99), 2)}')
print (f'Percentil 99 da nota em matemática de inscritos não treineiros: {round(np.quantile(nao_treineiros, 0.99), 2)}')

plt.hist(treineiros, bins=30, label='Treineiro', histtype='stepfilled', color='green')
plt.hist(nao_treineiros, bins=30, label='Não Treineiro', alpha=.7, histtype='stepfilled', color='red')
plt.xlabel('Nota em Matemática')
plt.ylabel('Quantidade')
plt.axvline(x=treineiros.mean(), color='limegreen')
plt.axvline(x=nao_treineiros.mean(), color='darkred')
plt.title('Distribuição das notas de Matemática dos inscritos treineiros e não treineiros')
plt.legend()
plt.show()

# Boxplot das notas de Matemática de treineiros e não treineiros
dict_notas = {'treineiros': treineiros.values.tolist(),
              'nao_treineiros': nao_treineiros.values.tolist()}
fig, ax = plt.subplots(figsize = (10, 10))
ax.boxplot(dict_notas.values())
ax.set_xticklabels(dict_notas.keys())
plt.title('Boxplot das notas de Matemática dos inscritos treineiros e não treineiros')
plt.show()

# Teste de hipótese
# Como o p-valor é próximo de 0, devemos rejeitar a hipótese nula (de que não existe correlação entre ser ou não treineiro na nota de Matemática no Enem)
# Desse modo, é possível concluir que o fato do inscrito ser treineiro está correlacionado com a sua nota de Matemática do Enem
stats.ks_2samp(treineiros, nao_treineiros)

## Modelagem
Nesta etapa é realizada a modelagem preditiva da nota das 5 provas utilizando a técnica Lasso

In [ ]:
# lista de variáveis explicativas para utilizar no modelo
lista_vars_explicativas = ['Q001','Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011','Q012', 'Q013',
                           'Q014', 'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021','Q022', 'Q023', 'Q024', 'Q025',
                           'IN_ACESSO', 'TP_ANO_CONCLUIU', 'TP_LINGUA', 'TP_SEXO', 'TP_DEPENDENCIA_ADM_ESC',
                           'NU_IDADE', 'CO_ESCOLA', 'CO_MUNICIPIO_PROVA', 'TP_ESCOLA', 'TP_COR_RACA', 
                           'NO_MUNICIPIO_NASCIMENTO', 'NO_MUNICIPIO_PROVA', 'TP_ST_CONCLUSAO', 'IN_LIBRAS',
                           'NO_MUNICIPIO_RESIDENCIA', 'TP_ENSINO', 'SG_UF_PROVA', 'TP_ESTADO_CIVIL',
                           'TP_NACIONALIDADE', 'IN_SEM_RECURSO', 'IN_SALA_ESPECIAL', 'SG_UF_NASCIMENTO',
                           'IN_TREINEIRO', 'IN_DEFICIT_ATENCAO', 'TP_SIT_FUNC_ESC', 'SG_UF_ESC',
                           'NO_MUNICIPIO_ESC', 'IN_LEDOR', 'CO_MUNICIPIO_ESC', 'IN_TEMPO_ADICIONAL',
                           'IN_DEFICIENCIA_AUDITIVA', 'TP_LOCALIZACAO_ESC', 'IN_DEFICIENCIA_MENTAL',
                           'IN_SURDEZ', 'IN_AUTISMO', 'IN_DEFICIENCIA_FISICA', 'IN_TRANSCRICAO', 'IN_MAQUINA_BRAILE',
                           'TP_PRESENCA_MT', 'TP_PRESENCA_LC', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_STATUS_REDACAO']
                           
                           
lista_df_submissao = []
lista_erros_treino = []
lista_erros_teste = []

# faz um modelo por target (nota da prova)
for target in lista_targets:
    print ('***************')
    print (target)
    print ('***************')

    # Separa dados em treino (80%) e teste (20%) - Hold out split
    df_train, df_test = train_test_split(df_desenv, test_size=0.2, random_state=42)

    # treino
    X_train = df_train[lista_vars_explicativas]
    y_train = df_train[target].copy()
    
    # teste
    X_test = df_test[lista_vars_explicativas]
    y_test = df_test[target].copy()
    
    # submit
    X_submit = df_submit[lista_vars_explicativas]

    # Lebel Encoder
    for coluna in X_train.columns.tolist():
      if (X_train[coluna].dtype != int and X_train[coluna].dtype != float):
        labelencoder = LabelEncoder()
        labelencoder.fit_transform(pd.concat([X_train[coluna], X_test[coluna], X_submit[coluna]], axis=0))
        X_train[coluna] = labelencoder.transform(X_train[coluna])
        X_test[coluna] = labelencoder.transform(X_test[coluna])
        X_submit[coluna] = labelencoder.transform(X_submit[coluna])

    # Remove valores missing
    X_train = X_train.fillna(-999)
    X_test = X_test.fillna(-999)
    X_submit = X_submit.fillna(-999)

    # Ignora notas missing
    y_train = y_train.fillna(0)
    y_test = y_test.fillna(0)

    #########
    # Lasso #
    #########
    
    clf = Lasso(alpha=1.0, max_iter = 10000)
    clf.fit(X_train, y_train)

    # importâncias das variáveis explicativas
    print_importancias_lasso(X_train, clf.coef_)

    # Erro de treino
    y_pred = clf.predict(X_train)
    erro = rmse_score(y_train, y_pred)
    print ('Erro de treino:', erro)
    lista_erros_treino.append(erro)

    # Erro de teste
    y_pred = clf.predict(X_test)
    erro = rmse_score(y_test, y_pred)
    print ('Erro de teste:', erro)
    lista_erros_teste.append(erro)
    
    # Previsão para o dataset de submissão
    df_submit_results = df_submit[['NU_INSCRICAO']].copy()
    df_submit_results[target] = clf.predict(X_submit)
    lista_df_submissao.append(df_submit_results)

    # limpa memória
    del df_train, df_test, X_train,  X_submit, X_test, y_train, y_test
    gc.collect()

### Resultados

In [ ]:
# junta predições em um único dataframe
print ('Erros de treino:', lista_erros_treino)
print ('Erros de teste:', lista_erros_teste)
print ('Erro médio de treino:', np.mean(lista_erros_treino))
print ('Erro médio de teste:', np.mean(lista_erros_teste))
df_submit_prediction = reduce(lambda left, right: pd.merge(left, right,on=['NU_INSCRICAO'], how='inner'), lista_df_submissao)
df_submit_prediction.to_csv(path_output_submission, sep=',', index=False)
print (df_submit_prediction.shape)
df_submit_prediction.head()

Erro de Treino: 78,9

Erro de Teste (hold out): 79,2

Erro de Teste (Leaderboard pública): 79,2